In [3]:
!ls -f /science/image/nlp-datasets/emanuele/data/flickr30k/

.			 resnet101_faster_rcnn_genome_imgfeats	X-101.lmdb
..			 bua-caffe-r101-fix36			annotations
X-101			 README					images
flickr30k-images.tar.gz  hard_negative.pkl
images.lst		 dataset_flickr30k.json


In [2]:
!ls -f /science/image/nlp-datasets/emanuele/data/flickr30k/X-101 | head

.
..
193082928_info.npy
2214821600_info.npy
4841988538.npy
5162402452_info.npy
4731694958.npy
129528391_info.npy
4683957629_info.npy
2609847254.npy
ls: write error: Broken pipe


In [4]:
!ls -f /science/image/nlp-datasets/emanuele/data/flickr30k/features/flickr30k_X101.lmdb | head

.
..
lock.mdb
data.mdb


In [7]:
!ls -f /science/image/nlp-datasets/emanuele/data/flickr30k/features/flickr30k_boxes36.lmdb

.  ..  lock.mdb  data.mdb


In [10]:
import lmdb
import pickle

In [9]:
features_path = "/science/image/nlp-datasets/emanuele/data/flickr30k/features/flickr30k_X101.lmdb"
env = lmdb.open(features_path, max_readers=1, readonly=True, lock=False, readahead=False, meminit=False)

In [14]:
with env.begin(write=False) as txn:
    _image_ids = pickle.loads(txn.get("keys".encode()))
_image_ids[:4]

[b'4841988538', b'4731694958', b'2609847254', b'47321520']

In [17]:
index = 0
image_id = _image_ids[index]
with env.begin(write=False) as txn:
    item = pickle.loads(txn.get(image_id))

In [19]:
item.keys()

dict_keys(['feature_path', 'features', 'image_height', 'image_width', 'num_boxes', 'objects', 'cls_prob', 'bbox'])

In [20]:
item['feature_path']

'4841988538'

In [22]:
item['bbox'][0]

array([ 99.64347,  68.05333, 193.47815, 133.9668 ], dtype=float32)

In [24]:
item['features'].astype('float32')

array([[1.3864238 , 0.        , 0.        , ..., 0.        , 3.7216337 ,
        2.0170946 ],
       [0.        , 1.4387386 , 0.        , ..., 1.9447222 , 0.        ,
        0.        ],
       [2.4254103 , 0.90743715, 0.        , ..., 0.46917439, 0.        ,
        0.        ],
       ...,
       [1.2574073 , 0.3921312 , 0.        , ..., 0.7907711 , 0.        ,
        0.        ],
       [0.        , 0.2256733 , 0.2729268 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.6976906 , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [30]:
import torch
from torch.nn import functional as F

In [28]:
item['features']

array([[1.3864238 , 0.        , 0.        , ..., 0.        , 3.7216337 ,
        2.0170946 ],
       [0.        , 1.4387386 , 0.        , ..., 1.9447222 , 0.        ,
        0.        ],
       [2.4254103 , 0.90743715, 0.        , ..., 0.46917439, 0.        ,
        0.        ],
       ...,
       [1.2574073 , 0.3921312 , 0.        , ..., 0.7907711 , 0.        ,
        0.        ],
       [0.        , 0.2256733 , 0.2729268 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.6976906 , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [31]:
att_feat = item['features'].astype('float32')
att_feat = torch.FloatTensor(att_feat)
att_feat = F.normalize(att_feat, dim=-1).numpy()
att_feat

array([[0.0190174 , 0.        , 0.        , ..., 0.        , 0.05104918,
        0.02766823],
       [0.        , 0.01649332, 0.        , ..., 0.02229379, 0.        ,
        0.        ],
       [0.03422999, 0.01280672, 0.        , ..., 0.00662149, 0.        ,
        0.        ],
       ...,
       [0.02113522, 0.00659116, 0.        , ..., 0.01329173, 0.        ,
        0.        ],
       [0.        , 0.00338306, 0.00409143, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03281737, 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [34]:
len(item['bbox'])

100

In [46]:
import h5py
from tqdm import tqdm

In [43]:
f = h5py.File("train.h5", "w")

In [44]:
f['image_id'] = _image_ids

In [50]:
num_boxes = []
wh = []
bbox = []
features = []
objects = []
with env.begin(write=False) as txn:
    for index, image_id in tqdm(enumerate(_image_ids), total=len(_image_ids)):
        item = pickle.loads(txn.get(image_id))
        num_boxes.append(item['num_boxes'])
        wh.append([(item['image_height'], item['image_width'])])
        bbox.append(item['bbox'])
        features.append(item['features'])
        objects.append(item['objects'])

100%|██████████| 33854/33854 [1:02:41<00:00,  9.00it/s]


In [51]:
f['num_boxes'] = num_boxes
f['wh'] = wh
f['bbox'] = bbox
f['features'] = features
f['objects'] = objects

In [52]:
f.close()

In [53]:
f = h5py.File("train.h5", "r")

In [55]:
list(f.keys())

['bbox', 'features', 'image_id', 'num_boxes', 'objects', 'wh']

In [56]:
len(f['bbox'])

33854

In [57]:
len(f['features'])

33854

In [58]:
len(f['image_id'])

33854

In [59]:
len(f['wh'])

33854

In [49]:
item.keys()

dict_keys(['feature_path', 'features', 'image_height', 'image_width', 'num_boxes', 'objects', 'cls_prob', 'bbox'])

In [36]:
ckpt = torch.load("/science/image/nlp-datasets/emanuele/checkpoints/M3P/checkpoint.pth", map_location="cpu")

In [37]:
ckpt.keys()

dict_keys(['epoch', 'n_total_iter', 'best_metrics', 'best_stopping_criterion', 'model', 'model_optimizer', 'params'])

In [40]:
# ckpt['model'].keys()

In [41]:
ckpt['epoch'], ckpt['n_total_iter']

(22, 28750)